In [9]:
import sys
sys.path.insert(0, "..")

from IPython.display import Markdown, display

from src.config.paths import DATA_DIR
import pymupdf4llm

In [ ]:
file_path = DATA_DIR / "pdf" / "example.pdf"

In [7]:
file_md = pymupdf4llm.to_markdown(file_path)